<a href="https://colab.research.google.com/github/MohamedElashri/HEP-ML/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Generating Theoretical particle physics abstracts

by [Mohamed Elashri](http://melashri.net)

Created: Dec 26th, 2020*
*Last updated: Dec 27th, 2020*



Generate particle physics paper abstracts using a pretrained neural network with a few lines of code using textgenrnn library. 

For more about textgenrnn, you can visit [Library GitHub repository](https://github.com/minimaxir/textgenrnn).



There are some combtablility issues between Tensorflow 2.x and textgenrnn so we will need to stick with version 1

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


I'm using google colab and it easier to upload files that contain abstract dataset for training our model . 

In [2]:
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Set the textgenrnn model configuration here: the default parameters here give good results for most workflows. (see the [Library demo notebook](https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb) for more information about these parameters)

If you are using an input file where documents are line-delimited, make sure to set `line_delimited` to `True`.

In [8]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 256,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 30,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 5000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 15,   # set higher to train the model for longer
    'gen_epochs': 1,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.1,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. 

From there you can upload files:

Upload **any text file** and update the file name in the cell below, then run the cell.

In [9]:
file_name = "df2.csv"
model_name = 'physics-extended'   # change to set file name of resulting trained models/texts

Let's do the training with aiming for a loss less than '1.0' to create sensible text consistently.

In [10]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 4-layer, 128-cell LSTMs
Training on 5,693,743 character sequences.
Epoch 1/5
5560/5560 [==============================] - 402s 72ms/step - loss: 1.4202
####################
Temperature: 0.2
####################
ticle part of the possibility of the model of the state of the stational potentials on the propagation of the stational potentials in the spectrum of the possibility of the spectrum of the stational part of the possibility of the state of the propagator of the possibility of the stationary condition

tions and the construction of the stational point functions and the properties of the state of the possibility of the standard model of the construction of the stationary conditions are also discussed. the spectrum of the spacetime of the state of the superspace of the spectrum of the phase transiti

s of the particle potential and the strong coupling in the part of the spacetime of the experimental constant and the string theory of the stational point propagat

You can download a large amount of generated text from your model with the cell below! Rerun the cell as many times as you want for even more text!

In [22]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 500 if model_cfg['word_level'] else 1000
  
timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We can download the weights and configuration files in the cell below, allowing you recreate the model on our own machines!

In [12]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

To recreate the model on your own computer, after installing textgenrnn and TensorFlow, you can create a Python script with:



```
from textgenrnn import textgenrnn
textgen = textgenrnn(weights_path='colaboratory_weights.hdf5',
                       vocab_path='colaboratory_vocab.json',
                       config_path='colaboratory_config.json')
 
textgen.generate_samples(max_gen_length=1000)
textgen.generate_to_file('textgenrnn_texts.txt', max_gen_length=1000)

```



---



# Appendix


If the model fails to load on a local machine due to a model-size-not-matching bug (common in >30MB weights), this is due to a file export bug from Colaboratory. To work around this issue, save the weights to Google Drive with the two cells below and download from there.

In [24]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [25]:
uploaded = drive.CreateFile({'title': '{}_weights.hdf5'.format(model_name)})
uploaded.SetContentFile('{}_weights.hdf5'.format(model_name))
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1EiXApEgD_iShMsXVqawpsY30U1g69dWP


If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1